# Multi Layer Perceptron

## Loading the dataset

In [1]:
from keras import models, layers, Input
import dask.dataframe as dd
import pandas as pd
import numpy as np
import logging
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

2024-10-20 11:12:42.462090: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#dtype_dict = {
#    # Columns that should be strings
#    'ResponseID': 'str',
#    'ExtendedSessionID': 'str',
#    'UserID': 'str',
#    'AttributeLevel': 'str',
#    'ScenarioTypeStrict': 'str',
#
#    # Columns that should be int8
#    'PedPed': 'int8',
#    'Barrier': 'int8',
#    'CrossingSignal': 'int8',
#    'NumberOfCharacters': 'int8',
#    'DiffNumberOFCharacters': 'int8',
#    'Man': 'int8',
#    'Woman': 'int8',
#    'Pregnant': 'int8',
#    'Stroller': 'int8',
#    'OldMan': 'int8',
#    'OldWoman': 'int8',
#    'Boy': 'int8',
#    'Girl': 'int8',
#    'Homeless': 'int8',
#    'LargeWoman': 'int8',
#    'LargeMan': 'int8',
#    'Criminal': 'int8',
#    'MaleExecutive': 'int8',
#    'FemaleExecutive': 'int8',
#    'FemaleAthlete': 'int8',
#    'MaleAthlete': 'int8',
#    'FemaleDoctor': 'int8',
#    'MaleDoctor': 'int8',
#    'Dog': 'int8',
#    'Cat': 'int8',
#    'Saved': 'int8',
#
#    # Columns that should be float32
#    'Finance_access': 'float32',
#    'ICT': 'float32',
#    'Industry_activity': 'float32',
#    'Overall_index': 'float32',
#    'Research_and_development': 'float32',
#    'Skills': 'float32',
#    'Total': 'float32',
#    'Males': 'float32',
#    'Females': 'float32',
#    'Passengers': 'float32',
#    'Pedestrians': 'float32'
#}

In [2]:
df = pd.read_csv('Data/cleaned_data.csv')

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,ResponseID,ExtendedSessionID,UserID,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,NumberOfCharacters,DiffNumberOFCharacters,Saved,Country,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills,Total,Males,Females,Passengers,Pedestrians
0,2223Xu54ufgjcyMR3,1425316635_327833569077076.0,327833569077076,0,1,0,Old,Age,5,0,0,MEX,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.60,0.55,0.80,0.60,0.5,0.40,13.60,22.049999,5.45,2.5024,3.8760
1,2223jMWDEGNeszivb,-1683127088_785070916172117.0,785070916172117,1,0,2,More,Utilitarian,5,2,0,CHE,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0.90,0.65,0.90,0.90,0.7,0.80,2.90,4.200000,1.60,0.5076,0.6237
2,222HpiEf2LtAwEg62,-1232628507_1597557389,1597557389,0,1,0,Female,Gender,2,0,0,UKR,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.75,0.50,0.65,0.65,0.5,0.65,13.25,21.750000,5.95,7.4120,5.6984
3,222KuWty7pNeiv77a,1654911454_3639764894860440.0,3639764894860440,1,0,0,Low,Social Status,2,0,0,USA,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0.90,0.65,0.80,1.00,1.0,0.75,12.50,17.850000,7.25,3.9603,1.9737
4,222LDp4wz24C3chzj,-1679158262_3623236506.0,3623236506,0,0,0,Fat,Fitness,2,0,0,DEU,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.80,0.80,0.90,0.90,0.8,0.75,4.20,6.350000,2.15,1.9120,0.6120


## Data preparation

In [5]:
# Delete the columns 'ResponseID', ExtendedSessionID' and 'UserID' and 'Country'
df = df.drop(['ResponseID', 'ExtendedSessionID', 'UserID', 'Country'], axis=1)

In [6]:
num_cols = ['NumberOfCharacters', 'DiffNumberOFCharacters', 'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat', 'Finance_access', 'ICT', 'Industry_activity', 'Overall_index', 'Research_and_development', 'Skills', 'Total', 'Males', 'Females', 'Passengers', 'Pedestrians']
cat_cols = ['AttributeLevel', 'ScenarioTypeStrict', 'CrossingSignal'] #Categorical columns that shouldn't be scaled: 'CrossingSignal'
binary_cols = ['PedPed', 'Barrier']

In [13]:
# Normalize the numerical columns
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [14]:
# Label encode the categorical columns
labelencoder = LabelEncoder()
for col in cat_cols:
    df[col] = labelencoder.fit_transform(df[col])

In [10]:
pd.set_option('display.max_columns', None)
df.head()

,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,NumberOfCharacters,DiffNumberOFCharacters,Saved,Man,Woman,Pregnant,Stroller,OldMan,OldWoman,Boy,Girl,Homeless,LargeWoman,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat,Finance_access,ICT,Industry_activity,Overall_index,Research_and_development,Skills,Total,Males,Females,Passengers,Pedestrians
0,0,1,0,9,0,1.00,0.0,0,0.0,0.0,0.0,0.0,0.4,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.60,0.526316,0.733333,0.60,0.5,0.40,0.365741,0.355216,0.197826,0.127978,0.216141
1,1,0,2,8,6,1.00,0.5,0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.90,0.631579,0.866667,0.90,0.7,0.80,0.035494,0.034173,0.030435,0.025960,0.028886
2,0,1,0,1,2,0.25,0.0,0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.473684,0.533333,0.65,0.5,0.65,0.354938,0.349820,0.219565,0.379065,0.321068
3,1,0,0,6,4,0.25,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.90,0.631579,0.733333,1.00,1.0,0.75,0.331790,0.279676,0.276087,0.202538,0.106614
4,0,0,0,0,1,0.25,0.0,0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,0.789474,0.866667,0.90,0.8,0.75,0.075617,0.072842,0.054348,0.097784,0.028212


## Train and evaluate the model

In [16]:
# Split dataset in features and target variable
feature_cols = ['NumberOfCharacters', 'DiffNumberOFCharacters', 'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat', 'Finance_access', 'ICT', 'Industry_activity', 'Overall_index', 'Research_and_development', 'Skills', 'Total', 'Males', 'Females', 'Passengers', 'Pedestrians']
label = 'Saved'
X = df[feature_cols] # Features
y = df[label] # Target variable

In [19]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size = 0.15, stratify=y, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size = 0.17647059, stratify=y_trainval, random_state = 42)
print(X_train.shape, X_val.shape, X_test.shape)

(6999994, 33) (1499999, 33) (1499999, 33)


In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(32, activation='relu',
                           input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = build_model()

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=150, batch_size=32, verbose=1)

/Users/timoklein/Library/CloudStorage/OneDrive-Personal/Universiteit/Studie/M_Data_Science_and_Society/year_2/Thesis/Code/timoklein/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
 56020/218750 ━━━━━━━━━━━━━━━━━━━━ 9:37 4ms/step - accuracy: 0.6534 - loss: 0.6224

In [ ]:
acc_history = history.history['accuracy']
val_acc_history = history.history['val_accuracy']

plt.plot(range(1, len(acc_history) + 1), acc_history)
plt.plot(range(1, len(val_acc_history) + 1), val_acc_history)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['training', 'validation'])
plt.show()

smooth_acc_history = smooth_curve(acc_history)
val_smooth_acc_history = smooth_curve(val_acc_history)

plt.plot(range(1, len(smooth_acc_history) + 1), smooth_acc_history)
plt.plot(range(1, len(val_smooth_acc_history) + 1), val_smooth_acc_history)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['training', 'validation'])
plt.show()

In [ ]:
# Training the final model and the whole training set
model = build_model()
model.fit(X_trainval, y_trainval,
          epochs=50, batch_size=32, verbose=0)
test_loss_score, test_acc_score = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc_score)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)
y_pred = (y_pred > 0.5)
print(y_pred)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)